In [ ]:
import mediapipe as mp
import cv2
import time
import numpy as np

In [ ]:
mpDraw = mp.solutions.drawing_utils
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,  # 偵測為靜態畫面，若為動態畫面（影片或攝影機）則設為真
               max_num_hands=2,                 # 最多偵測的手數量
               min_detection_confidence=0.9,    # 最小偵測置信度
               min_tracking_confidence=0.5)     # 最小追蹤置信度

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, # 最小偵測置信度
                    min_tracking_confidence=0.5)  # 最小追蹤置信度

## 身體骨架點

![](https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png)

## 手的骨架點

![](https://google.github.io/mediapipe/images/mobile/hand_landmarks.png)

In [ ]:
pTime = cTime = 0               # 計算FPS的初始參數

#cap = cv2.VideoCapture("MVI_4980.MP4")
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()                         # 讀取攝影機的有無訊息與當前畫面
    if not ret: break                               # 攝影機無畫面時關閉
    
    (imgHeight, imgWidth, _) = frame.shape          # 影像形狀的寬高(於後骨架需要正規化回來計算所需)
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # 彩色影像GBR通道轉換為RGB通道
    
    # 手骨架
    result = hands.process(imgRGB)                  # Mediapipe手骨架預測結果
    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)
            for i, lm in enumerate(handLms.landmark):
                
                # 手骨架點正規化還原
                xPos = int(lm.x * imgWidth)
                yPos = int(lm.y * imgHeight)
                
                # 影像上放置文字
                cv2.putText(frame, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)
                
                # 骨架點為8時：
                #if i==8:
                #    cv2.circle(frame, (xPos, yPos), 8, (255, 0, 0), cv2.FILLED) 
    
    # 身體骨架
    result = pose.process(imgRGB)                   # Mediapipe身體骨架預測結果
    if result.pose_landmarks:
        for i, lm in enumerate(result.pose_landmarks.landmark):
            mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
            # 身體骨架點正規化還原
            xPos = int(lm.x * imgWidth)
            yPos = int(lm.y * imgHeight)
            
            # 影像上放置文字
            cv2.putText(frame, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)
                

    # 計算FPS
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(frame, "FPS:%s" %(int(fps)), (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # 呈現影像
    cv2.imshow("Hand Tracking", frame)
    
    # 鍵盤按下q則關閉迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 關閉視窗與銷毀
cap.release()
cv2.destroyAllWindows()